In [1]:
import os
import pyspark
from pyspark.sql import SQLContext
import pyspark.sql.functions as F
import seaborn as sns
import matplotlib.pyplot as plt
import yaml

In [2]:
# Reads the YAML config file
def read_yaml_config():
    global cfg
    with open("configuration.yml", 'r') as ymlfile:
        cfg = yaml.load(ymlfile)

In [3]:
# Get Amazon Access Key Id
def get_amazon_access_key_id():
    if 'cfg' not in globals():
        read_yaml_config()
    return cfg["amazon"]["access_key_id"]

In [4]:
# Get Amazon Access Key Id
def get_amazon_secret_access_key():
    if 'cfg' not in globals():
        read_yaml_config()
    return cfg["amazon"]["secret_access_key"]

In [5]:
KEYSPACE = "sparkassandra"
APP_NAME = KEYSPACE
CASSANDRA_IP = "cassandra_node"

## Init PySpark

In [6]:
SUBMIT_ARGS = "--packages com.databricks:spark-csv_2.10:1.2.0,com.amazonaws:aws-java-sdk:1.10.34,org.apache.hadoop:hadoop-aws:2.6.0,com.datastax.spark:spark-cassandra-connector_2.10:1.6.0-M1 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [7]:
# Make sure the driver and workers all use Python2
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

In [8]:
#Init Spark Conf
conf = pyspark.SparkConf().setAppName(APP_NAME)
conf.set("spark.cassandra.connection.host",CASSANDRA_IP)

In [9]:
# Init Spark Context
sc = pyspark.SparkContext('spark://10.0.0.2:7077', conf = conf)

In [10]:
# Init Spark SQL Context
sql_ctx = SQLContext(sc)

In [11]:
hadoopConf = sc._jsc.hadoopConfiguration()
hadoopConf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")

hadoopConf.set("fs.s3n.awsAccessKeyId", get_amazon_access_key_id())
hadoopConf.set("fs.s3n.awsSecretAccessKey", get_amazon_secret_access_key())

## Create Cassandra Keyspace

https://github.com/dkoepke/cassandra-python-driver

## Create Table us_flights

## Create Table us_airports

## Load Airport file and store in Cassandra

In [12]:
airportRDD = sql_ctx.read.format('com.databricks.spark.csv').options(header='true').load('s3n://sparkassandra/airports.csv')

In [13]:
airportRDD.printSchema()

root
 |-- iata: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)



In [ ]:
airportRDD.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="airports", keyspace=KEYSPACE)\
    .save()

## Load US Flights files and store in Cassandra

In [14]:
flightsRDD = sql_ctx.read.format('com.databricks.spark.csv').options(header='true').load('s3n://sparkassandra/1987.csv')

In [15]:
# Add Id
flightsRDD = flightsRDD.withColumn("Id", F.concat_ws("-", flightsRDD["Year"], flightsRDD["Month"], flightsRDD["DayofMonth"], flightsRDD["DepTime"], flightsRDD["FlightNum"]));

In [16]:
for column in flightsRDD.columns:
    flightsRDD = flightsRDD.withColumnRenamed(column, column.lower());

In [17]:
flightsRDD.registerTempTable("us_flights")

In [18]:
flightsRDD.printSchema()

root
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- dayofmonth: string (nullable = true)
 |-- dayofweek: string (nullable = true)
 |-- deptime: string (nullable = true)
 |-- crsdeptime: string (nullable = true)
 |-- arrtime: string (nullable = true)
 |-- crsarrtime: string (nullable = true)
 |-- uniquecarrier: string (nullable = true)
 |-- flightnum: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- actualelapsedtime: string (nullable = true)
 |-- crselapsedtime: string (nullable = true)
 |-- airtime: string (nullable = true)
 |-- arrdelay: string (nullable = true)
 |-- depdelay: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- taxiin: string (nullable = true)
 |-- taxiout: string (nullable = true)
 |-- cancelled: string (nullable = true)
 |-- cancellationcode: string (nullable = true)
 |-- diverted: string (nullable = true)
 |-- carrierdelay:

In [19]:
flightsRDD.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="us_flights", keyspace=KEYSPACE)\
    .save()

Py4JJavaError: An error occurred while calling o80.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 6, platform_spark_worker_3.platform_spark_cluster_nw): java.io.InvalidClassException: org.apache.spark.sql.catalyst.expressions.Literal; local class incompatible: stream classdesc serialVersionUID = -4259705229845269663, local class serialVersionUID = 3305180847846277455
	at java.io.ObjectStreamClass.initNonProxy(ObjectStreamClass.java:616)
	at java.io.ObjectInputStream.readNonProxyDesc(ObjectInputStream.java:1630)
	at java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1521)
	at java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:1781)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1353)
	at java.io.ObjectInputStream.readArray(ObjectInputStream.java:1714)
	at java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1347)
	at 

In [ ]:
#sql_ctx.read.format("org.apache.spark.sql.cassandra").options(table="us_flights", keyspace=KEYSPACE).load().show()

TEST

In [ ]:
#RES = sql_ctx.read.format('com.databricks.spark.csv').options(header='true', delimiter=';').load('s3n://sparkassandra/file1.csv')
#RES.show()

In [ ]:
#RES.write\
#    .format("org.apache.spark.sql.cassandra")\
#    .mode('append')\
#    .options(table="bonhomme", keyspace=KEYSPACE)\
#    .save()